<a href="https://www.kaggle.com/code/kamaljp/handson-pyspark-notebook?scriptVersionId=121501818" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 15.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=331b59e53e8278adad49ff1ef10a2e5ebd17d177d1a1a85fcaab9dba6d8872ca
  Stored in directory: /root/.cache/pip/wheels/5a/54/9b/a89cac960efb57c4c35d41cc7c9f7b80daa21108bc376339b7
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests, json

from pyspark.sql import SparkSession

- We will start by going through the basics of Spark Sessions 

- Create simple Dataframes

- Understand how Spark Metastore works 

- Then bring in the data from the model API end point https://pyrite-ethereal-soccer.glitch.me/docs

- Then learn to do Transformation and loading into files and other locations

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    appName(f'{username} | Getting Started'). \
    master('yarn'). \
    getOrCreate()

In [3]:
spark = SparkSession. \
    builder. \
    appName('SparkOnKaggle'). \
    getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/09 01:18:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [5]:
#Creating spark objects from regular python objects

attendants = [(1, "Smith", "Deo", 1086.0, "Lineab"),
             (2, "Kenry", "Jord", 1950.0, "Irbin"),
              (3, "Roy", "Lyrd", 12357.0, "Irbin"),
              (4, "Bey", "Arod", 17510.0, "Lineab"),
             (5, "Kick", "Boxer", 7550.0, "Kingome"),
             (6, "Gill", "Nome", 1500.0, "New Wold"),
              (7, "Nill", "While", 6500.0, "New Wold"),
              (8, "Will", "Lobe", 18900.0, "Oreleans"),
              (9, "Shome", "Cole", 1980.0, "Kingome"),
              (10, "Pharel", "Phel", 75600.0, "Oreleans")
            ]

In [6]:
attendantSDF = spark.createDataFrame(attendants, schema="""attendant_id INT, f_name STRING,
                                     l_name STRING, salary FLOAT, Nationality STRING""")
attendantSDF.show()

+------------+------+------+-------+-----------+
|attendant_id|f_name|l_name| salary|Nationality|
+------------+------+------+-------+-----------+
|           1| Smith|   Deo| 1086.0|     Lineab|
|           2| Kenry|  Jord| 1950.0|      Irbin|
|           3|   Roy|  Lyrd|12357.0|      Irbin|
|           4|   Bey|  Arod|17510.0|     Lineab|
|           5|  Kick| Boxer| 7550.0|    Kingome|
|           6|  Gill|  Nome| 1500.0|   New Wold|
|           7|  Nill| While| 6500.0|   New Wold|
|           8|  Will|  Lobe|18900.0|   Oreleans|
|           9| Shome|  Cole| 1980.0|    Kingome|
|          10|Pharel|  Phel|75600.0|   Oreleans|
+------------+------+------+-------+-----------+



In [7]:
attendantSDF.collect()

[Row(attendant_id=1, f_name='Smith', l_name='Deo', salary=1086.0, Nationality='Lineab'),
 Row(attendant_id=2, f_name='Kenry', l_name='Jord', salary=1950.0, Nationality='Irbin'),
 Row(attendant_id=3, f_name='Roy', l_name='Lyrd', salary=12357.0, Nationality='Irbin'),
 Row(attendant_id=4, f_name='Bey', l_name='Arod', salary=17510.0, Nationality='Lineab'),
 Row(attendant_id=5, f_name='Kick', l_name='Boxer', salary=7550.0, Nationality='Kingome'),
 Row(attendant_id=6, f_name='Gill', l_name='Nome', salary=1500.0, Nationality='New Wold'),
 Row(attendant_id=7, f_name='Nill', l_name='While', salary=6500.0, Nationality='New Wold'),
 Row(attendant_id=8, f_name='Will', l_name='Lobe', salary=18900.0, Nationality='Oreleans'),
 Row(attendant_id=9, f_name='Shome', l_name='Cole', salary=1980.0, Nationality='Kingome'),
 Row(attendant_id=10, f_name='Pharel', l_name='Phel', salary=75600.0, Nationality='Oreleans')]

In [8]:
type(attendantSDF.collect()[0])

pyspark.sql.types.Row

In [9]:
from pyspark.sql.functions import *

In [10]:
#Three different ways to initiate transformation on the dataframes
#select
attendantSDF.select("f_name","l_name",concat('l_name','f_name').alias('full_name')).show()

+------+------+----------+
|f_name|l_name| full_name|
+------+------+----------+
| Smith|   Deo|  DeoSmith|
| Kenry|  Jord| JordKenry|
|   Roy|  Lyrd|   LyrdRoy|
|   Bey|  Arod|   ArodBey|
|  Kick| Boxer| BoxerKick|
|  Gill|  Nome|  NomeGill|
|  Nill| While| WhileNill|
|  Will|  Lobe|  LobeWill|
| Shome|  Cole| ColeShome|
|Pharel|  Phel|PhelPharel|
+------+------+----------+



In [11]:
#Using withColumn method
attendantSDF.withColumn('new_name',concat('Nationality',lit(' ** '),'f_name')).show()

+------------+------+------+-------+-----------+------------------+
|attendant_id|f_name|l_name| salary|Nationality|          new_name|
+------------+------+------+-------+-----------+------------------+
|           1| Smith|   Deo| 1086.0|     Lineab|   Lineab ** Smith|
|           2| Kenry|  Jord| 1950.0|      Irbin|    Irbin ** Kenry|
|           3|   Roy|  Lyrd|12357.0|      Irbin|      Irbin ** Roy|
|           4|   Bey|  Arod|17510.0|     Lineab|     Lineab ** Bey|
|           5|  Kick| Boxer| 7550.0|    Kingome|   Kingome ** Kick|
|           6|  Gill|  Nome| 1500.0|   New Wold|  New Wold ** Gill|
|           7|  Nill| While| 6500.0|   New Wold|  New Wold ** Nill|
|           8|  Will|  Lobe|18900.0|   Oreleans|  Oreleans ** Will|
|           9| Shome|  Cole| 1980.0|    Kingome|  Kingome ** Shome|
|          10|Pharel|  Phel|75600.0|   Oreleans|Oreleans ** Pharel|
+------------+------+------+-------+-----------+------------------+



In [12]:
#Using withColumn method
attendantSDF.selectExpr("salary * 2 AS new_salary", "CONCAT(f_name,l_name) AS new_name").show()

+----------+----------+
|new_salary|  new_name|
+----------+----------+
|    2172.0|  SmithDeo|
|    3900.0| KenryJord|
|   24714.0|   RoyLyrd|
|   35020.0|   BeyArod|
|   15100.0| KickBoxer|
|    3000.0|  GillNome|
|   13000.0| NillWhile|
|   37800.0|  WillLobe|
|    3960.0| ShomeCole|
|  151200.0|PharelPhel|
+----------+----------+



In [13]:
attendantSDF.drop('Nationality').show()

+------------+------+------+-------+
|attendant_id|f_name|l_name| salary|
+------------+------+------+-------+
|           1| Smith|   Deo| 1086.0|
|           2| Kenry|  Jord| 1950.0|
|           3|   Roy|  Lyrd|12357.0|
|           4|   Bey|  Arod|17510.0|
|           5|  Kick| Boxer| 7550.0|
|           6|  Gill|  Nome| 1500.0|
|           7|  Nill| While| 6500.0|
|           8|  Will|  Lobe|18900.0|
|           9| Shome|  Cole| 1980.0|
|          10|Pharel|  Phel|75600.0|
+------------+------+------+-------+



In [14]:
attendantSDF.show()

+------------+------+------+-------+-----------+
|attendant_id|f_name|l_name| salary|Nationality|
+------------+------+------+-------+-----------+
|           1| Smith|   Deo| 1086.0|     Lineab|
|           2| Kenry|  Jord| 1950.0|      Irbin|
|           3|   Roy|  Lyrd|12357.0|      Irbin|
|           4|   Bey|  Arod|17510.0|     Lineab|
|           5|  Kick| Boxer| 7550.0|    Kingome|
|           6|  Gill|  Nome| 1500.0|   New Wold|
|           7|  Nill| While| 6500.0|   New Wold|
|           8|  Will|  Lobe|18900.0|   Oreleans|
|           9| Shome|  Cole| 1980.0|    Kingome|
|          10|Pharel|  Phel|75600.0|   Oreleans|
+------------+------+------+-------+-----------+



In [15]:
attendantSDF.schema

StructType([StructField('attendant_id', IntegerType(), True), StructField('f_name', StringType(), True), StructField('l_name', StringType(), True), StructField('salary', FloatType(), True), StructField('Nationality', StringType(), True)])

In [16]:
type(attendantSDF.schema)

pyspark.sql.types.StructType

In [17]:
attendantSDF.printSchema()

root
 |-- attendant_id: integer (nullable = true)
 |-- f_name: string (nullable = true)
 |-- l_name: string (nullable = true)
 |-- salary: float (nullable = true)
 |-- Nationality: string (nullable = true)



In [18]:
attendantSDF.write.parquet(path='/kaggle/working/parquet',
                          mode='overwrite',
                          compression='snappy')

In [19]:
!ls -ltr /kaggle/working/parquet/

total 16
-rw-r--r-- 1 root root 1563 Mar  9 01:19 part-00003-237b6956-7c1b-4743-9f93-9af9a06df7b7-c000.snappy.parquet
-rw-r--r-- 1 root root 1463 Mar  9 01:19 part-00002-237b6956-7c1b-4743-9f93-9af9a06df7b7-c000.snappy.parquet
-rw-r--r-- 1 root root 1443 Mar  9 01:19 part-00001-237b6956-7c1b-4743-9f93-9af9a06df7b7-c000.snappy.parquet
-rw-r--r-- 1 root root 1451 Mar  9 01:19 part-00000-237b6956-7c1b-4743-9f93-9af9a06df7b7-c000.snappy.parquet
-rw-r--r-- 1 root root    0 Mar  9 01:19 _SUCCESS


In [20]:
new_parquet_read = spark.read.parquet('/kaggle/working/parquet')
new_parquet_read.show()

+------------+------+------+-------+-----------+
|attendant_id|f_name|l_name| salary|Nationality|
+------------+------+------+-------+-----------+
|           7|  Nill| While| 6500.0|   New Wold|
|           8|  Will|  Lobe|18900.0|   Oreleans|
|           9| Shome|  Cole| 1980.0|    Kingome|
|          10|Pharel|  Phel|75600.0|   Oreleans|
|           5|  Kick| Boxer| 7550.0|    Kingome|
|           6|  Gill|  Nome| 1500.0|   New Wold|
|           1| Smith|   Deo| 1086.0|     Lineab|
|           2| Kenry|  Jord| 1950.0|      Irbin|
|           3|   Roy|  Lyrd|12357.0|      Irbin|
|           4|   Bey|  Arod|17510.0|     Lineab|
+------------+------+------+-------+-----------+



In [21]:
attendantSDF.coalesce(1).write.mode('overwrite').option("compression","none"). \
    option('sep','|').format('csv').save('/kaggle/working/csv/')

In [22]:
!ls -ltr /kaggle/working/csv/

total 4
-rw-r--r-- 1 root root 277 Mar  9 01:19 part-00000-5f3fc0f0-62f4-47a8-8253-ca4e4583a3bc-c000.csv
-rw-r--r-- 1 root root   0 Mar  9 01:19 _SUCCESS


In [23]:
attendantSDF.write.mode('overwrite').option("compression","gzip"). \
    option('sep','|').format('csv').save('/kaggle/working/csv_gzip/')

In [24]:
!ls -ltr /kaggle/working/csv_gzip/

total 16
-rw-r--r-- 1 root root  71 Mar  9 01:19 part-00000-42dbb9ed-962b-4c63-9ce6-d1d82eb4eb88-c000.csv.gz
-rw-r--r-- 1 root root  71 Mar  9 01:19 part-00002-42dbb9ed-962b-4c63-9ce6-d1d82eb4eb88-c000.csv.gz
-rw-r--r-- 1 root root 113 Mar  9 01:19 part-00003-42dbb9ed-962b-4c63-9ce6-d1d82eb4eb88-c000.csv.gz
-rw-r--r-- 1 root root  68 Mar  9 01:19 part-00001-42dbb9ed-962b-4c63-9ce6-d1d82eb4eb88-c000.csv.gz
-rw-r--r-- 1 root root   0 Mar  9 01:19 _SUCCESS


In [25]:
temp = spark.read.csv('/kaggle/working/csv_gzip/',sep="|",inferSchema=True)

In [26]:
temp.show()

+---+------+-----+-------+--------+
|_c0|   _c1|  _c2|    _c3|     _c4|
+---+------+-----+-------+--------+
|  7|  Nill|While| 6500.0|New Wold|
|  8|  Will| Lobe|18900.0|Oreleans|
|  9| Shome| Cole| 1980.0| Kingome|
| 10|Pharel| Phel|75600.0|Oreleans|
|  5|  Kick|Boxer| 7550.0| Kingome|
|  6|  Gill| Nome| 1500.0|New Wold|
|  1| Smith|  Deo| 1086.0|  Lineab|
|  2| Kenry| Jord| 1950.0|   Irbin|
|  3|   Roy| Lyrd|12357.0|   Irbin|
|  4|   Bey| Arod|17510.0|  Lineab|
+---+------+-----+-------+--------+



In [27]:
full_read_csv = spark.read.csv('/kaggle/working/csv/',inferSchema=True,sep='|')
full_read_csv

DataFrame[_c0: int, _c1: string, _c2: string, _c3: double, _c4: string]

In [28]:
full_read_csv.show()

+---+------+-----+-------+--------+
|_c0|   _c1|  _c2|    _c3|     _c4|
+---+------+-----+-------+--------+
|  1| Smith|  Deo| 1086.0|  Lineab|
|  2| Kenry| Jord| 1950.0|   Irbin|
|  3|   Roy| Lyrd|12357.0|   Irbin|
|  4|   Bey| Arod|17510.0|  Lineab|
|  5|  Kick|Boxer| 7550.0| Kingome|
|  6|  Gill| Nome| 1500.0|New Wold|
|  7|  Nill|While| 6500.0|New Wold|
|  8|  Will| Lobe|18900.0|Oreleans|
|  9| Shome| Cole| 1980.0| Kingome|
| 10|Pharel| Phel|75600.0|Oreleans|
+---+------+-----+-------+--------+



!cat /kaggle/working/csv/part-00000-e8009cfb-463b-4b76-aae6-627a1bf84ccc-c000.csv.gz

### Introducing Spark Metastore

In [29]:
spark.conf.set("spark.sql.shuffle.partitions",'2')

In [30]:
spark.sql("DROP DATABASE IF EXISTS trial_database")
spark.sql("CREATE DATABASE trial_database")
spark.sql("USE DATABASE trial_database")

DataFrame[]

In [31]:
spark.catalog.currentDatabase()

'trial_database'

In [32]:
spark.catalog.listDatabases()

[Database(name='default', description='default database', locationUri='file:/kaggle/working/spark-warehouse'),
 Database(name='trial_database', description='', locationUri='file:/kaggle/working/spark-warehouse/trial_database.db')]

In [33]:
attendantSDF.write.saveAsTable("attendants", mode='overwrite',partitionBy='Nationality')

In [34]:
!ls /kaggle/working/spark-warehouse/trial_database.db

attendants


In [35]:
!ls /kaggle/working/spark-warehouse/trial_database.db/attendants

'Nationality=Irbin'    'Nationality=Lineab'    'Nationality=Oreleans'
'Nationality=Kingome'  'Nationality=New Wold'   _SUCCESS


In [36]:
!ls /kaggle/working/spark-warehouse/trial_database.db/attendants/'Nationality=Oreleans'

part-00003-a4024192-883c-4101-a8c6-00d6d344ca30.c000.snappy.parquet


In [37]:
!ls /kaggle/working/spark-warehouse/trial_database.db/attendants/'Nationality=New Wold'

part-00002-a4024192-883c-4101-a8c6-00d6d344ca30.c000.snappy.parquet
part-00003-a4024192-883c-4101-a8c6-00d6d344ca30.c000.snappy.parquet


In [38]:
spark.catalog.listTables()

[Table(name='attendants', database='trial_database', description=None, tableType='MANAGED', isTemporary=False)]

In [39]:
spark.sql('DESCRIBE FORMATTED attendants').show(truncate=False)

+----------------------------+-----------------------------------------------------------------+-------+
|col_name                    |data_type                                                        |comment|
+----------------------------+-----------------------------------------------------------------+-------+
|attendant_id                |int                                                              |null   |
|f_name                      |string                                                           |null   |
|l_name                      |string                                                           |null   |
|salary                      |float                                                            |null   |
|Nationality                 |string                                                           |null   |
|# Partition Information     |                                                                 |       |
|# col_name                  |data_type                

In [40]:
spark.catalog.createTable("attendants_new_csv",
                                  path='/kaggle/working/csv/', 
                                 source='csv',
                                 sep='|',
                                 inferSchema='true')

DataFrame[_c0: int, _c1: string, _c2: string, _c3: double, _c4: string]

In [41]:
spark.sql('DESCRIBE FORMATTED attendants_new_csv').show(truncate=False)

+----------------------------+----------------------------+-------+
|col_name                    |data_type                   |comment|
+----------------------------+----------------------------+-------+
|_c0                         |int                         |null   |
|_c1                         |string                      |null   |
|_c2                         |string                      |null   |
|_c3                         |double                      |null   |
|_c4                         |string                      |null   |
|                            |                            |       |
|# Detailed Table Information|                            |       |
|Database                    |trial_database              |       |
|Table                       |attendants_new_csv          |       |
|Created Time                |Thu Mar 09 01:19:19 UTC 2023|       |
|Last Access                 |UNKNOWN                     |       |
|Created By                  |Spark 3.3.2       

In [42]:
spark.sql("SELECT * FROM attendants").show()

+------------+------+------+-------+-----------+
|attendant_id|f_name|l_name| salary|Nationality|
+------------+------+------+-------+-----------+
|           2| Kenry|  Jord| 1950.0|      Irbin|
|           7|  Nill| While| 6500.0|   New Wold|
|           5|  Kick| Boxer| 7550.0|    Kingome|
|           9| Shome|  Cole| 1980.0|    Kingome|
|           8|  Will|  Lobe|18900.0|   Oreleans|
|          10|Pharel|  Phel|75600.0|   Oreleans|
|           6|  Gill|  Nome| 1500.0|   New Wold|
|           1| Smith|   Deo| 1086.0|     Lineab|
|           3|   Roy|  Lyrd|12357.0|      Irbin|
|           4|   Bey|  Arod|17510.0|     Lineab|
+------------+------+------+-------+-----------+



In [43]:
spark.sql(f"DROP DATABASE IF EXISTS trial_database CASCADE")

DataFrame[]

In [44]:
spark.catalog.listDatabases()

[Database(name='default', description='default database', locationUri='file:/kaggle/working/spark-warehouse')]

### Starting the ETL process from an API

https://pyrite-ethereal-soccer.glitch.me/docs

https://hugovk.github.io/top-pypi-packages/top-pypi-packages-30-days.json

In [45]:
url = 'https://hugovk.github.io/top-pypi-packages/top-pypi-packages-30-days.json'
res = requests.request(method='GET',url=url)
data_json = res.json()

In [46]:
data_json

{'last_update': '2023-03-01 09:13:26',
 'query': {'bytes_billed': 535241424896,
  'bytes_processed': 535240895791,
  'cached': False,
  'estimated_cost': '2.44'},
 'rows': [{'download_count': 550614649, 'project': 'boto3'},
  {'download_count': 300695650, 'project': 'urllib3'},
  {'download_count': 277634197, 'project': 'requests'},
  {'download_count': 264559611, 'project': 'botocore'},
  {'download_count': 260816531, 'project': 'setuptools'},
  {'download_count': 240160757, 'project': 'charset-normalizer'},
  {'download_count': 234354685, 'project': 'google-api-core'},
  {'download_count': 224752673, 'project': 'idna'},
  {'download_count': 223028824, 'project': 'certifi'},
  {'download_count': 210742541, 'project': 's3transfer'},
  {'download_count': 192310456, 'project': 'python-dateutil'},
  {'download_count': 186983236, 'project': 'typing-extensions'},
  {'download_count': 170886826, 'project': 'six'},
  {'download_count': 169141864, 'project': 'pyyaml'},
  {'download_count': 152

In [47]:
main_data = data_json['rows']

In [48]:
spark.stop()

In [49]:
etl_spark = SparkSession. \
    builder. \
    appName('ETL_spark'). \
    getOrCreate()

In [50]:
etl_spark

In [51]:
api_datadf = etl_spark.createDataFrame(data=main_data)

In [52]:
api_datadf.schema

StructType([StructField('download_count', LongType(), True), StructField('project', StringType(), True)])

In [53]:
api_datadf.printSchema()

root
 |-- download_count: long (nullable = true)
 |-- project: string (nullable = true)



In [54]:
api_datadf.count()

5000

In [55]:
# A bit more involved api.. using the pyrite api
pyriteurl = "https://pyrite-ethereal-soccer.glitch.me/biker/v1/deliveryd"
pyriteRes = requests.request(method='GET',url=pyriteurl)
pyriteData = pyriteRes.json()
pyriteData

[{'ID': '0x2318',
  'Delivery_person_ID': 'COIMBRES13DEL01',
  'Type_of_Vehicle': 'electric_scooter',
  'DeliveryPersonAge': 29.5,
  'DeliveryPersonRatings': 4.6,
  'TypeOfVehicle': 2.0,
  'DeliveryPersonID': 2.0},
 {'ID': '0x3474',
  'Delivery_person_ID': 'BANGRES15DEL01',
  'Type_of_Vehicle': 'motorcycle',
  'DeliveryPersonAge': 28.0,
  'DeliveryPersonRatings': 4.6,
  'TypeOfVehicle': 0.0,
  'DeliveryPersonID': 266.0}]

In [56]:
pyriteSDF = etl_spark.createDataFrame(data = pyriteData)
pyriteSDF.show()

+-----------------+----------------+---------------------+------------------+------+-------------+----------------+
|DeliveryPersonAge|DeliveryPersonID|DeliveryPersonRatings|Delivery_person_ID|    ID|TypeOfVehicle| Type_of_Vehicle|
+-----------------+----------------+---------------------+------------------+------+-------------+----------------+
|             29.5|             2.0|                  4.6|   COIMBRES13DEL01|0x2318|          2.0|electric_scooter|
|             28.0|           266.0|                  4.6|    BANGRES15DEL01|0x3474|          0.0|      motorcycle|
+-----------------+----------------+---------------------+------------------+------+-------------+----------------+



In [57]:
pyriteSDF.printSchema()

root
 |-- DeliveryPersonAge: double (nullable = true)
 |-- DeliveryPersonID: double (nullable = true)
 |-- DeliveryPersonRatings: double (nullable = true)
 |-- Delivery_person_ID: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- TypeOfVehicle: double (nullable = true)
 |-- Type_of_Vehicle: string (nullable = true)



In [58]:
#Writing the data to spark catalog
etl_spark.sql("CREATE DATABASE IF NOT EXISTS api_database")
etl_spark.sql("USE DATABASE api_database")
etl_spark.catalog.listDatabases()

[Database(name='api_database', description='', locationUri='file:/kaggle/working/spark-warehouse/api_database.db'),
 Database(name='default', description='default database', locationUri='file:/kaggle/working/spark-warehouse')]

In [59]:
pyriteSDF.write.saveAsTable("pyrite_table")
etl_spark.sql("SELECT * FROM pyrite_table").show(truncate=False)

+-----------------+----------------+---------------------+------------------+------+-------------+----------------+
|DeliveryPersonAge|DeliveryPersonID|DeliveryPersonRatings|Delivery_person_ID|ID    |TypeOfVehicle|Type_of_Vehicle |
+-----------------+----------------+---------------------+------------------+------+-------------+----------------+
|29.5             |2.0             |4.6                  |COIMBRES13DEL01   |0x2318|2.0          |electric_scooter|
|28.0             |266.0           |4.6                  |BANGRES15DEL01    |0x3474|0.0          |motorcycle      |
+-----------------+----------------+---------------------+------------------+------+-------------+----------------+



In [60]:
etl_spark.sql("SHOW TABLES").show()

+------------+------------+-----------+
|   namespace|   tableName|isTemporary|
+------------+------------+-----------+
|api_database|pyrite_table|      false|
+------------+------------+-----------+



In [61]:
etl_spark.sql("SHOW VIEWS").show()

+---------+--------+-----------+
|namespace|viewName|isTemporary|
+---------+--------+-----------+
+---------+--------+-----------+



In [62]:
etl_spark.sql("SHOW DATABASES").show()

+------------+
|   namespace|
+------------+
|api_database|
|     default|
+------------+



In [63]:
etl_spark.catalog.listTables()

[Table(name='pyrite_table', database='api_database', description=None, tableType='MANAGED', isTemporary=False)]

In [64]:
pyriteSDF.coalesce(1).write.csv(path='/kaggle/working/pyrite_csv',
                               sep='|',header=True,
                               mode='overwrite')

In [65]:
!ls /kaggle/working/pyrite_csv/

_SUCCESS  part-00000-67ef24b4-95ad-4b23-b47f-d5a97ad9127e-c000.csv


In [66]:
!cat /kaggle/working/pyrite_csv/part-00000-d97727fc-3917-464b-9379-cf80127126eb-c000.csv

cat: /kaggle/working/pyrite_csv/part-00000-d97727fc-3917-464b-9379-cf80127126eb-c000.csv: No such file or directory
